In [1]:
from threading import Thread
import concurrent.futures
import time

import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json
import requests

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,processDatesIT
from FMPGet import FMPGet

In [2]:
a = FMPGet(apikey)

In [3]:
a.run(verbose=True)

G1_MI_MAJORINDEXES SUCCESS
G1_CAL_EARNINGSCALENDAR_UPC SUCCESS
G1_MI_SP500LIST SUCCESS
G1_MI_NASDAQLIST SUCCESS
G1_MI_SP500HIST SUCCESS
G1_MI_DOWJLIST SUCCESS
G1_MI_DOWJHISTLIST SUCCESS
G1_MI_NASDAQHISTLIST SUCCESS
G1_COM_SYMBOLS SUCCESS
G1_INST_CIKLIST SUCCESS
G1_COM_PRICES SUCCESS
G1_CV_DELISTED SUCCESS
G1_ETF_SYMBOLS SUCCESS
G1_CV_ETFS SUCCESS
G1_ETF_PRICES SUCCESS
G1_MF_SYMBOLS SUCCESS
G1_CV_SYMBOLS SUCCESS
G1_MF_PRICES SUCCESS
G1_CV_TRADABLE SUCCESS
Retrieving dow-specific tickerset(funds,ciks,indexes,etc)


AttributeError: module 'pandas.io.sql' has no attribute 'read'

In [7]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [8]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [9]:
! rm ../../FinancesTEST.db

rm: ../../FinancesTEST.db: No such file or directory


In [10]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [11]:
#conn = sqlite3.connect('../../Finances.db')
conn = sqlite3.connect('FinancesTEST.db')
c = conn.cursor()

In [13]:
def load_url(pair,apikey):
    name = pair[0]
    url = pair[1]
    tab = get_appended(url,apikey,name)
    return tab

In [14]:
%%time
rate_limit = 10
offset = 1/rate_limit
max_workers = rate_limit -1
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url,pair,apikey): pair for pair in independents.iteritems()}
    for future in concurrent.futures.as_completed(future_to_url):
        time.sleep(offset)
        pair = future_to_url[future]
        try:
            tab = future.result()
        except Exception as exc:
            print(pair,'FAILED',exc)
            
        else:
            c.execute('drop table if exists '+ pair[0])
            tab.to_sql(pair[0],con=conn,index=False,if_exists='append')
            print(pair, 'SUCCESS')

('G1_MI_MAJORINDEXES', 'https://financialmodelingprep.com/api/v3/quotes/index?apikey=') SUCCESS
('G1_MI_SP500HIST', 'https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey=') SUCCESS
('G1_CAL_EARNINGSCALENDAR_UPC', 'https://financialmodelingprep.com/api/v3/earning_calendar?apikey=') SUCCESS
('G1_MI_SP500LIST', 'https://financialmodelingprep.com/api/v3/sp500_constituent?apikey=') SUCCESS
('G1_MI_NASDAQLIST', 'https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=') SUCCESS
('G1_MI_DOWJLIST', 'https://financialmodelingprep.com/api/v3/dowjones_constituent?apikey=') SUCCESS
('G1_MI_DOWJHISTLIST', 'https://financialmodelingprep.com/api/v3/historical/dowjones_constituent?apikey=') SUCCESS
('G1_COM_SYMBOLS', 'https://financialmodelingprep.com/api/v3/symbol/available-commodities?apikey=') SUCCESS
('G1_MI_NASDAQHISTLIST', 'https://financialmodelingprep.com/api/v3/historical/nasdaq_constituent?apikey=') SUCCESS
('G1_COM_PRICES', 'https://financialmodelingprep.c

In [ ]:
#setup database
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = ['000'+str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs_c + indexes + ciks + funds + commods))

In [ ]:
#setup subset of tickers
dow_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_DOWJLIST;',con=conn).symbol)
nasdaq_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_NASDAQLIST;',con=conn).symbol)
sp500_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

In [ ]:
#define subset to work with
stocksub = dow_tickers
#stocksub = sp500_tickers
indexsub = ['^IXIC','^GSPC','^DJI']
#indexsub = ['^IXIC','^GSPC','^DJI','^VIX','^RUA','^FTSE','^NDX','^N225','GDAXI']

In [ ]:
#get top US etfs proportionate to stock list
num_etfs = int(np.ceil(len(stocksub)/10))
profile = get_appended(dependents['P1b1000_CV_PROFILE'],apikey,'P1b1000_CV_PROFILE',tickers = etfs_c)
etfsub = list(profile[profile.country == 'US'].sort_values(by='volAvg',ascending=False).symbol[:num_etfs+1])
tickersub = stocksub + etfsub

In [ ]:
#get mutual fund data
fundtable = get_appended(dependents['T1_INST_MUTFUNDHOLDERS'],apikey,'T1_INST_MUTFUNDHOLDERS',tickers=tickersub,verbose=False)
fundsub = list(fundtable.holder.value_counts().index)

In [ ]:
mfs = pd.io.sql.read_sql('select * from G1_MF_SYMBOLS;',con=conn)
fundsub = list(np.random.choice(list(mfs[mfs.name.isin(fundsub)].symbol),int(np.ceil(len(stocksub)/3))))

In [ ]:
#establish subset of commods
#commodsub = commods[:3]
commodsub = commods

In [ ]:
#get ciks of institutional holders for ciksub
instable = get_appended(dependents['T1_INST_HOLDERS'],apikey,'T1_INST_HOLDERS',tickers=tickersub)
instsub = list(instable.holder.value_counts().index)
instsub = list(np.random.choice(instsub,500)) #account for many missing inst-cik mappings

In [ ]:
#retrieve ciks
ciklist = pd.io.sql.read_sql('select * from G1_INST_CIKLIST',con=conn)
ciksub = list(ciklist[ciklist.name.apply(lambda x: x.lower()).isin([i.lower() for i in instsub])].cik)
ciksub = ['000'+str(c) for c in (ciksub)]
ciksub = [c for c in ciksub if len(c)==10]

In [ ]:
#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocksub + etfsub + commodsub + fundsub,
     'P':stocksub + etfsub + fundsub,
     'T':stocksub + etfsub,
     'S':stocksub,
     'E':etfsub,
     'C':commodsub,
     'F':ciksub,
     'I':indexsub,
     'M':fundsub}

In [ ]:
%%time
schemas = open("schema.sql", "a+")
for name, url in dependents.iteritems():
    
    tablename = '_'.join(name.split('_')[1:])
    print(tablename)
    
    if name[0] == 'U':
            continue 
    elif name[0]=='G': #if ticker info not needed
        continue
    else:
        tab = get_appended(url,apikey,name,tickers=d.get(name[0]),verbose=True)
    
    if len(tab) == 0: 
        print(tablename,' FAILED')
        #log error
        continue
    
    #load sql
    a = pd.io.sql.get_schema(tab,tablename)
    c.execute('drop table if exists '+ tablename)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(tablename,con=conn,index=False,if_exists='append')